## @yullano90 | Scripts Python e R no Power Query

# TE LIGA:

In [ ]:
# Tudo funcionará apenas se em sua máquina os interpretadores R e Python estiverem instalados belezãã?.
# Na dúvida, vai lá no "python.org" /ou no "cram.r-project.org" (use o R 3.5 que é filé e sem bugs).
# Sempre acompanhe os fóruns e a negada da comunidade pra saber a versão que roda fino. Fichãã?

----------------------------------------------------------------------------------------------
# Importante lembrar que o conceito de Portabilidade com R e Python não é suportado no Microsoft Excel.
# Portanto, se você portar um código M no Excel, ao rodar "Python.Execute() não vai rolar gompáde!"
# Pelo menos até o momento em que subo esse notebook não há suplementação para isso.
----------------------------------------------------------------------------------------------

In [ ]:
#Carregue a consulta a seguir e nomeie como "TableScripts":

------------------------------------------------------------------------------------------------------------
let
    TabelaScripts =
        Table.FromRows(Json.Document(Binary.Decompress(Binary.FromText(
        "jVLNSsQwEH6V0HPEJmn6c1wWhBW8iZeyh7iN0qXdlGS74PN4EBc8iU+QFzMzab
        1oF6GdfDP55j91nWwGf75a9crqzpDVTjW6f0lo8se3pXVyY2yv/Jt/NeRWndQy8
        U4drf/YjZ1yZDCW9P7syMGQwba9bq0hTvfaHa0mjSY85Ww51PLN2n827bMh69E6
        E4wPqjM2nHt1uGZVAE/6FEHoLwL1aGdLG8F+RDLE40HlIqMyD6BIgyhBVCDmn6U
        pckXAIpM0L8BJgibxOgg58cADuFlAmcxpUUJI9IRUTGI+cACGFEiG3GVR0bSaeV
        LMPIbq5AZkhkZBs5/cHIQAkWEp0TYVvTzKe3NUHewCZ+nAk2IHF8mbg9vZNryHL
        x3XPOiwddOYSOZTRTg3Uc41cWyNFXl8Kf69adU/ov2qZPsN",
        BinaryEncoding.Base64), Compression.Deflate)),
        let _t = ((type nullable text) meta [Serialized.Text = true])
        in type table [
                C1 = _t, C2 = _t, C3 = _t, C4 = _t, C5 = _t, C6 = _t, C7 = _t, C8 = _t
                ]
        ),

 //Etapas de Transformação dos dados:
        RemoveUpLines = Table.Skip(TabelaScripts,4),
        RemoveDownLines = Table.RemoveLastN(RemoveUpLines,4),
        Headers = Table.PromoteHeaders(RemoveDownLines, [PromoteAllScalars=true]),
        Unpivot = Table.Unpivot(Headers,
        {"jan/19", "fev/19", "mar/19", "abr/19", "mai/19", "jun/19"}, "Atributo", "Valor.1"),
        AlterType2 = Table.TransformColumnTypes(Unpivot,{{"Atributo", type date}}),
        RenameC = Table.RenameColumns(AlterType2,{
            {"Valor.1", "Matrículas"}, {"Atributo", "Data"}
                }
            )
in
    RenameC
------------------------------------------------------------------------------------------------------------


### Exemplo de Script Python

In [ ]:
# No pandas temos os nossos queridos Dataframes, no Power Query temos o "Dataset".
# Importante usar o método .reset_index() para resetar a indexação automática.

# Faça referência ao objeto "TabelaScripts" e adicione o script abaixo:

-------------------------------------------------------------------------------------
   df_python = dataset.groupby(["Código Curso"]).mean().reset_index()
-------------------------------------------------------------------------------------


In [ ]:
#O código M após a inserção do Script pode ficar da seguinte forma:

---------------------------------------------------------------------------------------------------------------------
let
    Fonte = TableScripts,
    Script_Python = Python.Execute(
        "df_python = dataset.groupby([""Código Curso""]).mean().reset_index()",[dataset=Fonte]
    ),
    df_python = Script_Python{[Name="df_python"]}[Value]
in
    df_python
---------------------------------------------------------------------------------------------------------------------



### Exemplo de Script R

In [ ]:
# Faça referência ao objeto "TabelaScripts" e adicione o script abaixo:
-------------------------------------------------------------------------------------
df_R <- aggregate(dataset$"Matrículas", by=list(dataset$"Código Curso"), FUN=mean)
-------------------------------------------------------------------------------------


In [ ]:
#O código M após inserção do Script R ficará da seguinte forma:

---------------------------------------------------------------------------------------------------------------------
let
    Fonte = TableScripts,
    Script_R = R.Execute(
        "df_R <- aggregate(dataset$""Matrículas"", by=list(dataset$""Código Curso""), FUN=mean)",[dataset=Fonte]
    ),
    df_R = Script_R{[Name="df_R"]}[Value]
in
    df_R
---------------------------------------------------------------------------------------------------------------------
